In [1]:
import pandas as pd
df = pd.read_excel("GooglePlaystore.xlsx")

In [2]:
outlier_index = df[df["Reviews"] == "3.0M"].index
df = df.drop(outlier_index)

In [3]:
varies_with_device_index = df[df.eq("Varies with device").any(axis=1)].index
df = df.drop(varies_with_device_index)

In [4]:
df['Android Ver'] = df['Android Ver'].str.replace(r'\s+and\s+up|\s+\-\s+\d+(\.\d+)*', '').str.extract(r'(\d+(\.\d+)?)').astype(float)[0]

In [5]:
df['Installs'] = df['Installs'].str.replace(',', '').str.rstrip('+').astype('int64')
df = df[df['Installs'].apply(lambda x: isinstance(x, int))]

In [6]:
missing_rating_mask = df['Rating'].isnull() & (df['Reviews'] < 100) & (df['Installs'] < 50000)
df = df[~missing_rating_mask]

In [7]:
average_rating_by_category = df.groupby('Category')['Rating'].transform('mean')
df['Rating'] = df['Rating'].fillna(average_rating_by_category.round(2))

In [8]:
df['Size'] = df['Size'].str.replace('M', 'e6').str.replace('k', 'e3').str.replace(r'[^\d\.]', '', regex=True).astype(float).astype(int)

In [9]:
df.to_excel("result1.xlsx", index=False)